In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from models import *
from review_dataset import ReviewDataset
from torch.utils.data import DataLoader
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, ndcg_score

### Main

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
args = {
    'device' : device,
    'epochCount':50,
    'batch_size': 64,
    'ini_rev_dim': 768,
    'emb_dim': 512, 
    'num_heads': 1,
    'train_threshold': 0.6,
    'test_threshold': 0.7
    }

train_dataset = ReviewDataset(target='train')
train_loader = DataLoader(train_dataset, batch_size=args["batch_size"], shuffle=True)
test_dataset = ReviewDataset(target='test')
test_loader = DataLoader(test_dataset, batch_size=args["batch_size"], shuffle=True)

# Initialize Model
ini_rev_dim, emb_dim, num_heads = args['ini_rev_dim'], args['emb_dim'], args['num_heads']
nbr_module = Integrated_Neighbor_module(ini_rev_dim=ini_rev_dim, emb_dim=emb_dim, num_heads=num_heads).to(device)

# Loss criteria
criterion = nn.BCELoss().to(device)
optimizer = torch.optim.Adam(nbr_module.parameters(), lr=1e-4, weight_decay=1e-5)

### Train

In [10]:
pbar = trange(args['epochCount'])
train_loss_list, train_acc_list, train_precision_list, train_recall_list, train_f1_list = [], [], [], [], []
test_loss_list, test_acc_list, test_precision_list = [], [], []
for epoch in pbar:
    
    nbr_module.train()
    train_loss, train_acc, train_precision, train_recall, train_f1 = [], [], [], [], []
    train_precision, train_recall = [], []
    # batches_pbar = tqdm(train_loader, desc='Training...', colour='blue',leave=False)
    for batch in train_loader:
        user_data, item_data, _, y = batch
        y = y.to(device)
        user_emb, user_itemEmb = user_data['user_emb'], user_data['user_itemEmb']
        user_revEmb, user_nbr_revEmb = user_data['user_revEmb'], user_data['user_nbr_revEmb']

        item_emb, item_userEmb = item_data['item_emb'], item_data['item_userEmb']
        item_revEmb, item_nbr_revEmb = item_data['item_revEmb'], item_data['item_nbr_revEmb']
        
        output_logits = nbr_module(user_emb.to(device), user_itemEmb.to(device), user_nbr_revEmb.to(device),
                                user_revEmb.to(device), item_revEmb.to(device), item_emb.to(device),
                                item_nbr_revEmb.to(device), item_userEmb.to(device) )
        
        # train model
        loss = criterion( torch.squeeze(output_logits, dim=1), y.float() )
        # Gradients stored in the parameters in the previous step should be cleared out first
        optimizer.zero_grad()
        # Compute the gradients for parameters
        loss.backward()
        # grad_norm = nn.utils.clip_grad_norm_(nbr_module.parameters(), max_norm=10)  試試有無影響
        # Update the parameters with computed gradients
        optimizer.step()
        result_logits = torch.where( output_logits > args['train_threshold'], 1, 0 ).squeeze(dim=1)

        acc = (result_logits == y).float().mean()
        precision = precision_score(y.cpu(), result_logits.cpu(), zero_division=0)
        recall = recall_score(y.cpu(), result_logits.cpu(), zero_division=0)
        f1 = f1_score(y.cpu(), result_logits.cpu(), zero_division=0)

        train_loss.append(loss.item())
        train_acc.append(acc.cpu())
        train_precision.append(precision)
        train_recall.append(recall)
        train_f1.append(f1)

    train_loss_list.append( np.mean(train_loss) )
    train_acc_list.append( np.mean(train_acc) )
    train_precision_list.append( np.mean(train_precision) )
    train_recall_list.append( np.mean(train_recall) )
    train_f1_list.append( np.mean(train_f1) )

    # train result
    # print(f"Train | loss = {train_t_loss:.5f}, acc = {train_t_acc:.4f}, precision = {train_t_precision:.4f}, recall = {train_t_recall:.4f}, f1 = {train_t_f1}")

    # ---------- Validation ----------
    nbr_module.eval()
    test_loss, test_acc, test_f1 = [], [], []
    test_precision, test_recall = [], []
    for batch in test_loader:
        user_data, item_data, _, y = batch
        y = y.to(device)
        user_emb, user_itemEmb = user_data['user_emb'], user_data['user_itemEmb']
        user_revEmb, user_nbr_revEmb = user_data['user_revEmb'], user_data['user_nbr_revEmb']

        item_emb, item_userEmb = item_data['item_emb'], item_data['item_userEmb']
        item_revEmb, item_nbr_revEmb = item_data['item_revEmb'], item_data['item_nbr_revEmb']
        
        with torch.no_grad():
            output_logits = nbr_module(user_emb.to(device), user_itemEmb.to(device), user_nbr_revEmb.to(device),
                                    user_revEmb.to(device), item_revEmb.to(device), item_emb.to(device),
                                    item_nbr_revEmb.to(device), item_userEmb.to(device) )
            
            loss = criterion( torch.squeeze(output_logits, dim=1), y.float() )
            result_logits = torch.where( output_logits > args['test_threshold'], 1, 0 ).squeeze(dim=1)

            acc = (result_logits == y).float().mean()
            precision = precision_score(y.cpu(), result_logits.cpu(), zero_division=0)
            recall = recall_score(y.cpu(), result_logits.cpu(), zero_division=0)
            f1 = f1_score(y.cpu(), result_logits.cpu(), zero_division=0)

            test_loss.append(loss.item())
            test_acc.append(acc.cpu())
            test_precision.append(precision)
            test_recall.append(recall)
            test_f1.append(f1)
    
    test_loss_list.append(np.mean(test_loss))
    test_acc_list.append(np.mean(test_acc))
    test_precision_list.append(np.mean(test_precision))
    pbar.set_postfix( result= f"loss: {np.mean(test_loss):.5f}, acc: {np.mean(test_acc):.3f},\
    precision: {np.mean(test_precision):.3f}, recall: {np.mean(test_recall):.3f}, f1: {np.mean(test_f1):.3f}" )

100%|██████████| 50/50 [27:57<00:00, 33.54s/it, result=loss: 0.56552, acc: 0.750,    precision: 0.835, recall: 0.794, f1: 0.812]


In [11]:
# 結果寫入 excel
note = 'neg_sample, BCE_Loss, lr4, 2 FC(Tanh)'
problem = ''
res_df = pd.read_csv('result_plot/results.csv')
# result
train_res = [args['epochCount'], 'train', args['batch_size'], np.mean(train_loss_list[-10:]), np.mean(train_acc_list[-10:]),
             np.mean(train_precision_list[-10:]), np.mean(train_recall_list[-10:]), np.mean(train_f1_list[-10:]), note, problem ]
temp = pd.DataFrame(train_res).T
temp.columns = ['epoch', 'type', 'batch_size', 'Loss', 'Acc', 'Precision', 'Recall', 'F1', 'Note', 'Problem']
res_df = pd.concat([res_df, temp], axis=0, ignore_index=True)
res_df.to_csv('result_plot/results.csv', index=False)
res_df.tail()

,epoch,type,batch_size,Loss,Acc,Precision,Recall,F1,Note,Problem
6,50,train,64,0.345558,0.840221,0.892363,0.876115,0.882569,"neg_sample, BCE_Loss, lr4, 2 FC(Tanh)",NaN
7,50,train,64,0.711222,0.886682,0.906094,0.96097,0.93198,"BCE_w, lr4, 2 FC(Tanh)",NaN
8,50,train,64,0.300555,0.86221,0.907771,0.925411,0.915484,"BCE_Loss, lr4, 2 FC(Tanh)",NaN
9,50,train,64,0.528859,0.857003,0.880499,0.919345,0.898181,"neg_sample, BCE_w, lr4, 2 FC(Tanh)",NaN
10,50,train,64,0.347047,0.839402,0.89133,0.87593,0.881962,"neg_sample, BCE_Loss, lr4, 2 FC(Tanh)",


In [ ]:
def draw_curve(train_loss, plot_name, f_name):
    plt.plot(train_loss, color="blue", label="Train")
    plt.legend(loc="upper right")
    plt.title(plot_name)
    plt.savefig(f'result_plot/excel_plot/{f_name}.png')
    plt.show()

idx = len(res_df) + 1
draw_curve(train_loss_list, 'Train Loss_batch'+str(args['batch_size']), str(idx)+'_train_loss')
draw_curve(train_acc_list, 'Train Acc_batch'+str(args['batch_size']), str(idx)+'_train_acc')
draw_curve(test_loss_list, 'Test Loss_batch'+str(args['batch_size']), str(idx)+'_test_loss')
draw_curve(test_acc_list, 'Test Acc_batch'+str(args['batch_size']), str(idx)+'_test_acc')
draw_curve(test_precision_list, 'Test Pre_batch'+str(args['batch_size']), str(idx)+'_test_pre')
# torch.save(nbr_module.state_dict(), 'model/parameters/nbr_module_lr4')